In [6]:
import requests
from bs4 import BeautifulSoup
import json
# ------- STAGE 1 -------
# Interaction Code
url = 'https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=15174;type=tournament'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [7]:
# Parser Code
links = []
table_rows = soup.select('table.engineTable > tbody > tr.data1')
for row in table_rows:
    match_url = "https://www.espncricinfo.com" + row.select('td')[6].select('a')[0]['href']
    links.append(match_url)
# Next Stag

In [8]:
bowling_summary = []

In [9]:
for url in links:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    match = soup.find('span', text='MATCH FLOW').parent.parent.next_sibling.find_all_next('div')
    team1 = match[1].select('span')[0].text.replace(" innings","")
    team2 = match[0].select('span')[0].text.replace(" innings","")
    match_info = team1 + ' Vs ' + team2   
    tables = soup.select('div > table.ds-table')
    first_inning_rows = [row for row in tables[1].select('tbody > tr') if len(row.select('td')) >= 11]
    second_inning_rows = [row for row in tables[3].select('tbody > tr') if len(row.select('td')) >=11]
    summary = []
    try:
        
       for i in range(0,len(first_inning_rows)):
            row = first_inning_rows[i]

            cols = row.select('td')
            summary.append({
                'match': match_info,
                'bowlingTeam': team1,
                'bowlerName': cols[0].select('a > span')[0].text,
                'overs': cols[1].text,
                'maiden': cols[2].text,
                'runs': cols[3].text,
                'wickets': cols[4].text,
                'economy': cols[5].text,
                '0s': cols[6].text,
                '4s': cols[7].text,
                '6s': cols[8].text,
                'wides': cols[9].text,
                'noBalls': cols[10].text
            })
           
           
    except Exception as e:
        print(e)
    
    try:        
        for i in range(0,len(second_inning_rows)):
            row = second_inning_rows[i]
            cols = row.select('td')
            summary.append({
                'match': match_info,
                'bowlingTeam': team2,
                'bowlerName': cols[0].select('a > span')[0].text,
                'overs': cols[1].text,
                'maiden': cols[2].text,
                'runs': cols[3].text,
                'wickets': cols[4].text,
                'economy': cols[5].text,
                '0s': cols[6].text,
                '4s': cols[7].text,
                '6s': cols[8].text,
                'wides': cols[9].text,
                'noBalls': cols[10].text
            })
       
    except Exception as e:
        print(e)
    bowling_summary.append(summary)

In [10]:
with open('../WPL_json_files/bowling_summary.json', 'w') as f:
    json.dump(bowling_summary, f, indent=4)